# Preregistration: Urn Task with Confidence

## A. Hypotheses

1. There exists subclinical variation in psychiatric traits that are associated with differences in metacognition performance separate from task performance.
2. Individuals using strategies with higher information complexity exhibit lower metacognitive performance.
3. Individuals using strategies with higher distance to bound exhibit lower metacognitive performance.

## B. Methods

### Participants

We will recruit 500? participants on Prolific [[Rouault et al needed 470 to assess confidence vs depression with 80% power using similar questionnaire length. we can ballpark or do a power analysis]]. In order to be recruited for the task, participants must satisfy the following inclusion criteria: must have completed at least 20 tasks and have an at least 95% task approval rating. Of the participants who begin the task, we apply the following exclusion criteria: exclude participants who do not complete the entire experiment including questionnaire, fail at least one of the attention check questions in the questionnaire where they are instructed which option to select, [[maybe something about accuracy or not always selecting same responses in task/questionnaire]].

### Study design

Participants will perform 1000 trials of a bead estimation and confidence judgement task. On each trial, participants are shown a black or white bead that was drawn from one of two jars. Both jars contain only black and white beads, and participants are informed that the left jar contains 80% black beads and 20% white beads, and that the right jar contains 20% black beads and 80% white beads (flipped in the diagram below). The jar the next bead comes from has a 90% chance of remaining the same as the current jar, and a 10% chance of switching to the opposite jar. This switching probability, or hazard rate, is disclosed to the participant. Their task is to predict the color of the next bead that is drawn and to give a binary confidence judgement on their prediction. Thus, on any trial, their options are: black bead with high confidence, black bead with low confidence, white bead with high confidence, or white bead with low confidence. 

<center><img src="./Figs/HiddenMarkovTask-14.png" alt="Drawing" style="width: 700px;"/></center>

The normative strategy in this task is to use the history of past beads to infer which of the two jars is most likely to generate the next bead. Participants are instructed that it is best to guess the next jar that will be used. They are also asked to try to answer high confidence roughly half the time and low confidence roughly half the time.

The sequences of beads (and their underlying jars) observed by each participant is identical. We use the bead sequence, jar sequence, participant bead choices, and participant confidence judgements in the information bottleneck and regression analyses described in Section C.

### Questionnaire

After performing the behavioral task, subjects completed a questionnaire on Qualtrics assessing a range of psychiatric symptoms. This questionnaire included the following 8 self-report surveys:
* Depression Anxiety Stress Scales (DASS)
* Short Scales for Measuring Schizotypy (SSMS) 
* Urgency, Premeditation (lack of), Perseverance (lack of), Sensation Seeking, Positive Urgency, Impulsive Behavior Scale (UPPS)
* Alcohol Use Disorders Identification Test (AUDIT)
* Liebowitz Social Anxiety Scale (LSAS)
* Obsessive-Compulsive Inventory - Revised (OCI-R)
* Apathy Evaluation Scale (AES)
* Eating Attitudes Test (EAT)

In addition, cognitive motivation was assessed using the 18-item Need for Cognition Scale (NCS) as a proxy for IQ.

Two attention check questions were included at approximately the halfway point and close to the end of the questionnaire for a total of 200 questions. Each survey is coded following the recommended scheme to yield a total score for each psychiatric symptom. These aggregated scores, as well as the individual question responses, are normalized with respect to the responses of the rest of the participants to be used in the factor analysis and regression described in Section C.

This questionnaire was inspired by a similar one used by Rouault et. al. 2018.

## C. Analysis Plan

### Information Bottleneck

We estimate the amount of past information subjects use to make decisions and their predictive accuracy using the information-bottleneck method (Tishby et. al. 2000) on behavioral choice data. This method assumes that subjects form an internal model $M$ that uses past observations ($X_{past}$) to predict some future aspect of the task ($X_{future}$). One benefit of this method is that it allows for the quantification of information complexity without requiring specific assumptions or constraints on the model $M$.

We compute the mutual information between $X_{past}$ and $M$ as a measure of the amount of information $M$ encodes about $X_{past}$. We call this $I_{past} := I(X_{past}, M)$. We compute the mutual information between $M$ and $X_{future}$ as a measure of the predicive accuracy of $M$. We call this $I_{future} := I(M, X_{future})$. Larger values of $I_{past}$ and $I_{future}$ correspond to models with higher information complexity and higher predictive accuracy, respectively (Filipowicz et. al. 2020).

Both $I_{past}$ and $I_{future}$ can be calculated using only the bead response or using the bead response and confidence judgement together. In the first case, we will use bead choices $R$ as proxies for the participant's mental models $M$. In the second, we will use the combined bead choices and confidence judgements $(R, C)$ as proxies for their mental models $M$. This requires the assumptions that responses are veridical representations of subject mental models and that their mental models are stable throughout the period being measured.

Next, we use the history of beads observed by the participant as $X_{past}$. Due to the limits of human working memory, we are likely able to capture a substantial portion of strategies with reasonably limited window sizes into the past. In addition, our hazard rate serves as a temporal filter, so that more distant past observations hold less information about the future. Thus, although an infinite window into the past would yield the most precise $I_{past}$, we can approximate it well with a limited window size and improve the computational feasibility. Using pilot data from six subjects, we can see how $I_{past}$ evolves with increasing window sizes into the past:

<center><img src="./Figs/Ipast_window.png" alt="window" style="width: 500px;"/></center>

Moreover, we run into undersampling problems when we compute empirical estimates of the information bottleneck bound using the embo package for window sizes above 6. Thus, for our estimates of subject information complexity and the empirical information bottleneck bound, we limit X_past to the previous 6 beads that were observed.

Finally, we define $X_{future}$ to be the sequence of colors of the next bead.

Further details, including the definition of the function $I()$ and code for the empirical estimate of the bound can be found at [[cite Alex's paper that goes with the prereg]] and Piasini et. al. 2021.

In addition to computing $I_{past}$ and $I_{future}$ for both the bead choice only and the bead choice and confidence judgement pair, we also calculate a "distance to bound" ($DTB$) reflecting the optimality of the model. We consider a model to be optimal for a given information complexity if the $I_{future}$ lies on the information bottleneck bound for the given $I_{past}$. Thus, we can measure the vertical distance, or difference in $I_{future}$, between a participant's model and the empirical bound.

We use the following three values from the information bottleneck analyses as task-related variables in the regression analysis:
1. $I_{past\,:\,R}$
2. $DTB_R$
3. $\frac{I_{future\,:\,(R,C)}}{I_{future\,:\,R}}$

### Linear Regressions
We fit linear regressions to examine the relationships between task-related variables and psychiatric symptoms, age, and cognitive motivation. These task-related variables include the accuracy, the mean trial-by-trial confidence, and the information bottleneck parameters: $I_{past}$ and distance to bound for the bead choices only, and the ratio between $I_{future}$ for the bead choice and confidence judgement pair and $I_{future}$ for the bead choice only. We include the log-transformed total score for each psychiatric questionnaire, age, cognitive motivation, and gender as fixed effects. Because of the high correlation between different psychiatric symptom questionnaires, we perform separate regressions for each symptom, controlling for age, cognitive motivation, and gender. All regressors are normalized to allow for comparison of regression coefficients. Following the syntax of the lm function from the stats package in R, we fit:

<center>Task-related variable ~ Psychiatric symptom + Age + Cognitive motivation + Gender,</center>
    
for each task-related variable and psychiatric symptom tested in our questionnaire.

We also perform linear regressions without the psychiatric symptom variables to assess how task performance is associated with age and cognitive motivation:

<center>Task-related variable ~ Age + Cognitive motivation + Gender.</center>

We perform multiple hypotheses corrections over the psychiatric symptom questionnaires using the Benjamini-Hochberg method.

### Factor Analysis

Next, we perform maximum likelihood factor analysis to elucidate latent symptom dimensions that can account for a majority of the variance across the item-level psychiatric symptom scores. Similar to analyses performed in Gillan et. al. (2016) and Rouault et. al. (2018), we use the psych package in R (fa() function), with an oblique rotation (oblimin). First, the hetcor function from the polycor R package is used to compute the correlatio matrix between the 180 individual questionnaire items. Then, we select the number of latent factors using Cattell's criterion (Cattell 1966), as implemented in the nFactors R package (nCng function). An example on pilot data from five subjects is shown below:

<center><img src="./Figs/CNG.png" alt="CNG" style="width: 400px;"/></center>

We label the latent factors by examining the loadings across questionnaire items. Continuing with the example from above, we obtain the following loadings:

<center><img src="./Figs/factorloadings_pilot.png" alt="Factor Loadings" style="width: 800px;"/></center>

Finally, we also examine the association between these latent factors and the task-related variables. We perform similar regression analyses to those outlined in the Linear Regressions section, fitting the following model:
<center>Task-related variable ~ Each latent factor + Age + Cognitive motivation + Gender.</center>

## D. References

Tishby, N., Pereira, F. C., and Bialek, W., (2000). The information bottleneck method. *ArXiv Preprint Physics*, 1–16. DOI: 10.1108/eb040537

[[probably also all the questionnaires]]

Filipowicz, A.L.S., Levine, J., Piasini, E., Tavoni, G., Kable, J.W., and Gold, J.I., (2020). The comparable strategic flexibility of model-free and model-based learning. *bioRxiv*. DOI: 10.1101/2019.12.28.879965

Piasini, E., Filipowicz, A.L.S., Levine, J., and Gold, J.I., (2021). Embo: a Python package for empirical data analysis using the Information Bottleneck. *Journal of Open Research Software*, **9**(1), p.10. DOI: 10.5334/jors.322


Rouault, M., Seow, T., Gillan, C. M., and Fleming, S. M., (2018). Psychiatric Symptom Dimensions Are Associated With Dissociable Shifts in Metacognition but Not Task Performance. *Biological psychiatry*, **84**(6), 443–451. DOI: 10.1016/j.biopsych.2017.12.017

Gillan C.M., Kosinski M., Whelan R., Phelps E.A., and Daw N.D., (2016). Characterizing a psychiatric symptom dimension related to deficits in goal-directed control. *eLife*, **5**. DOI: 10.7554/eLife.11305 

Cattell R.B., (1966). The scree test for the number of factors. *Multivariate Behav Res*. **1**, 245–276. DOI: 10.1207/s15327906mbr0102_10